<a href="https://colab.research.google.com/github/nik1806/Semantic-segmentation/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mounting google drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [2]:
# Load the data and organizing it 
# root_dir = 'drive/My Drive/'
# data_dir = root_dir + 'NN_pro_data'

In [3]:
# change directory
# import os
# os.chdir(data_dir)
# !ls

In [4]:
# unzip data 
#!unzip gtFine_trainvaltest.zip
# !unzip leftImg8bit_trainvaltest.zip

In [5]:
# !ls

In [6]:
# Insert the directory - FOR importing user-defined modules/packages
# import sys
# sys.path.insert(0,'/content/drive/My Drive/Semantic-segmentation')

### Import libraries

In [1]:
# from dataset import cityscapes_train
# from dataset import cityscapes_val
from torchvision import transforms
from torchvision.datasets import Cityscapes
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from datasets.custom_transforms import decode_segmap, unNormalize

### Dataset reader 

In [2]:
#from utils.custom_transforms import pair_transform_train, pair_transform_val
#data_dir = '../datasets/cityscapes/'
#dst_train = Cityscapes(data_dir, split='train', mode='fine', target_type='semantic', transforms=pair_transform_train) # data_dir
#dst_val = Cityscapes(data_dir, split='val', mode='fine', target_type='semantic', transforms=pair_transform_val)
## data length
#length_train = len(dst_train)
#length_val = len(dst_val)
#print(length_train)
#print(length_val)

In [3]:
from datasets import cityscapes

In [4]:
# data directory path
data_dir = '../datasets/cityscapes/'
base_size = 288
crop_size = 256

In [5]:
# dataset readers
dst_train = cityscapes.CityscapesSegmentation(crop_size, base_size, root=data_dir, split='train')
dst_val = cityscapes.CityscapesSegmentation(crop_size, base_size, root=data_dir, split='val')

Found 2975 train images
Found 500 val images


### Visualize the data

In [6]:
#from utils.custom_transforms import unNormalize

In [7]:
## Verifying transformations (Especially consistancy among image and segmentation map)

# # img, segmap = dst_train[16]
# img, segmap = dst_val[21]

# fig = plt.figure(figsize=(10, 7))

# fig.add_subplot(1,2,1)
# img = img.data.numpy() # data in image and current form of matrix
# img = unNormalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # unNormalize
# img = img.transpose((1,2,0)).astype(np.uint8) # change dtype to correct format for display
# plt.title('Original')
# plt.imshow(img) # original
# plt.axis('off')

# # ground truth
# fig.add_subplot(1, 2, 2)
# label = segmap.data.numpy() # data in image and current form of matrix
# # label = decode_segmap(label)
# plt.title('Ground truth')
# plt.imshow(label) 
# plt.axis('off')

In [8]:
#np.unique(label)

### Dataloader

In [9]:
bs = 10
cls_num = 19
#Dataloaders
train_loader = DataLoader(dst_train, batch_size=bs, shuffle=True)
val_loader = DataLoader(dst_val, batch_size=bs,shuffle=False)

In [10]:
# dataiter = iter(train_loader)
# #print(dataiter.next())
# (images, labels) = dataiter.next()

# dataiter_val = iter(val_loader)
# (images_val, labels_val) = dataiter_val.next()
# labels_val.shape

### Model

In [11]:
from models.networkT2 import R2UNet

In [12]:
device = torch.device("cuda:6")
model = R2UNet(in_channels=3,n_classes=cls_num).to(device)

### Hyper parameters

In [13]:
epochs = 30
lr = 3e-4
wdk = 5e-4 #earlier 1e-4

In [14]:
from torch.optim import Adam, lr_scheduler
criterion = torch.nn.CrossEntropyLoss(size_average=True, ignore_index=255)
optimizer = Adam(model.parameters(), lr, (0.9, 0.999),  eps=1e-08, weight_decay=wdk) 

/opt/conda/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


### Training model

In [ ]:
loss_train = []
loss_val = []

for epoch in range(epochs):
    #Training
    model.train()
    running_loss = 0

    #for i, (images, labels) in enumerate(train_loader):
    for i, samples in enumerate(train_loader):

        #inputs = images.to(device)
        #labels = labels.to(device)
        inputs = samples['image'].to(device)
        labels = samples['label'].to(device).long()
        ## labels = labels.squeeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)

     
    #loss = criterion(outputs,targets[:,0])
        #loss = criterion(outputs,labels[:,0].long())
        # loss = criterion(outputs,labels.long())
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        ###accumulating loss for each batch
        running_loss += loss.item()

        if i%60 == 0:
            print("Epoch{}, iter{}, running loss: {}".format(epoch, i, running_loss/(bs*(i+1))))
                  
    loss_train.append(running_loss/len(train_loader))

    print("Epoch{}, Training loss: {}".format(epoch, running_loss/len(train_loader)))
    torch.save(model.state_dict(), f'../weights/T2/epoch_{epoch}.pth')

    #Validation
    model.eval()
    running_loss_val = 0
    for i, samples in enumerate(val_loader):
        inputs = samples['image'].to(device)
        labels = samples['label'].to(device).long()
        # labels = labels.squeeze(1)
             
        with torch.no_grad(): 
            outputs = model(inputs)
            # loss = criterion(outputs,labels.long())
            loss = criterion(outputs,labels)

            ###accumulating loss for each batch
            running_loss_val += loss.item()


        #if i%10 == 0:
    loss_val.append(running_loss_val/len(val_loader))
    print("Epoch{}, Validation loss: {}".format(epoch, running_loss_val/len(val_loader)))


Epoch0, iter0, running loss: 0.5798403739929199
Epoch0, iter60, running loss: 0.19672707182462099
Epoch0, iter120, running loss: 0.15734175793395555
Epoch0, iter180, running loss: 0.1401642770727695
Epoch0, iter240, running loss: 0.13154896371592129
Epoch0, Training loss: 1.2511200074781508
Epoch0, Validation loss: 1.7938858342170716
Epoch1, iter0, running loss: 0.10356841087341309
Epoch1, iter60, running loss: 0.09139333680027821
Epoch1, iter120, running loss: 0.08974680565605479
Epoch1, iter180, running loss: 0.08900963160214503
Epoch1, iter240, running loss: 0.08731157139129164
Epoch1, Training loss: 0.8713546127280933
Epoch1, Validation loss: 0.7993522143363953
Epoch2, iter0, running loss: 0.07310233116149903
Epoch2, iter60, running loss: 0.07915941994698321
Epoch2, iter120, running loss: 0.08045784823658053
Epoch2, iter180, running loss: 0.07934139545451212
Epoch2, iter240, running loss: 0.07924820778280868
Epoch2, Training loss: 0.7800984491637889
Epoch2, Validation loss: 0.97661

In [ ]:
# x = [i for i in range(len(train_loader))]
x = range(1, epochs+1)
plt.title("Plot showing training and validation loss against number of epochs")
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.plot(x, loss_train, color='b', label='Training loss')
plt.plot(x, loss_val, color='r', label='Validation loss')
plt.legend()
plt.savefig('results/T2_loss_curves.png', bbox_inches='tight')
plt.show()
# plt.plot()

### Visualize results 

In [ ]:

# epochs=1
# move model to gpu
model = model.to(device)
# load the model states
model.load_state_dict(torch.load(f'../weights/T2/epoch_{epochs-1}.pth'))

# model in evaluation model -> batchnorm, dropout etc. adjusted accordingly
model.eval()
# iterator on training data
data = iter(train_loader)
# init figure object
fig = plt.figure(figsize=(10,40))
pred_rgb = list()
# for img, label in trainloader:
for i in range(10):
    sample = next(data) # next batch
    imgs, labels = sample['image'], sample['label']
    # img, label = img.to(device).unsqueeze(0), label.to(device) # to gpu
    # using just one image
    img = imgs[0].to(device).unsqueeze(0) # to gpu & add dummy batch dim
    # gnd = np.asarray(label[0]) 
    # deactivate autograd engine - reduce memory usage 
    with torch.no_grad(): 
        pred = model(img) # forward pass
        # output of model is orderedDict
#         pred = pred['out'] # 21(class)xHxW
        pred = pred.squeeze(0)
        
        # extract most probable class through C-dim 
        pred_label = torch.argmax(pred, dim=0).cpu().numpy()
        # convert labels to color code
#         pred_rgb = dst_train.decode_segmap(pred_label)
        pred_rgb = decode_segmap(pred_label, nc=cls_num, dataset='cityscapes')

        # plotting
        # original image
        fig.add_subplot(10, 3, 3*i+1)
        img = imgs[0].data.cpu().numpy() # data in image and current form of matrix
        img = unNormalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # unNormalize
        img = img.transpose((1,2,0)).astype(np.uint8) # change dtype to correct format for display
        plt.title('Original')
        plt.imshow(img) # original
        plt.axis('off')
        # ground truth
        fig.add_subplot(10, 3, 3*i+2)
        label = labels[0].data.numpy() # data in image and current form of matrix
        label = decode_segmap(label, nc=cls_num, dataset='cityscapes')
#         label = dst_train.decode_segmap(label)
        plt.title('Ground truth')
        plt.imshow(label) 
        plt.axis('off')
        # prediction
        fig.add_subplot(10, 3, 3*i+3)
        plt.title('Prediction')
        plt.imshow(pred_rgb.astype(np.uint8))
        plt.axis('off')

plt.savefig('results/T2 results.png', bbox_inches='tight')    
plt.plot()





### Evaluation and plotting 


In [ ]:
#5 evaluation matrices to be used: sensitivity, specificity, accuracy, AUC, and DC
from utils.eval_metrics import dice_coefficient_custom, roc_auc_custom, accuracy_se_sp_custom, sensitivity_custom, specificity_custom# user defined
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, multilabel_confusion_matrix

def evaluate_batch(gnd_b, pred_b, cls:int = 34):
    """
        Calculate evalution scores over the batch.
    Args:   
        gnd_b: BxHxW tensor; ground truth labels; each element of matrix in B dim contains class label from (0-20)
        pred_b: BxCxHxW tensor; each element contains predicted class label 
                here C=21 (0-20; no. of classes); each C corresponds to probabilites for that class,
                eg. C=0 contain score at each element in matrix HxW 
    Return:
        f1_score, auc_score, dice_coeeficient (averaged over batch size)
    """
    # to cpu and as numpy ndarray
    gnd_b = gnd_b.cpu()

    batch_size = gnd_b.shape[0]
    
    # extract most probable class through C-dim 
    label_b = torch.argmax(pred_b, dim=1).cpu()
    sensitivity = specificity = accuracy = auc = dice = 0
    # iterate over batch elements
    for i in range(batch_size):
        gnd = gnd_b[i,:,:] 
        label = label_b[i,:,:]
        #f1 += f1_score(gnd.flatten(), label.flatten(), average='micro')
        #sensitivity += sensitivity_custom(gnd, label)
        
        
        #specificity += specificity_custom(gnd, label)
        #accuracy += accuracy_se_sp_custom(gnd, label)
        temp = accuracy_se_sp_custom(gnd, label)
        accuracy += temp[0]
        sensitivity += temp[1]
        specificity += temp[2]
        # auc += roc_auc_score(gnd.flatten(), label.flatten(), average='micro', multi_class='ovr')
        auc += roc_auc_custom(gnd.numpy(), label.numpy(), cls, average='micro')
        dice += dice_coefficient_custom(gnd.numpy(), label.numpy(), cls)

    return [sensitivity/batch_size, specificity/batch_size, accuracy/batch_size, auc/batch_size, dice/batch_size]

In [ ]:
# list of score 
sensitivity_list = []
specificity_list = []
accuracy_list = []
auc_list = []
dice_list = []

# For print
print('Epochs\t Sensitivity-score Specificity-score Accuracy-score ROC-AUC-score\t Dice score')

# move model to gpu
model = model.to(device)
# loop for original number of epochs
for i in range(epochs):
    # load the model states
    model.load_state_dict(torch.load(f'../weights/T2/epoch_{i}.pth'))
    # model in evaluation model -> batchnorm, dropout etc. adjusted accordingly
    model.eval()
    # evaluation score variables to store values over each epoch
    sensitivity_score = specificity_score = accuracy_score = auc_score = dice_score = 0


    for img, label in val_loader:
        img, label = img.to(device), label.to(device) # to gpu
        # deactivate autograd engine - reduce memory usage 
        with torch.no_grad(): 
            pred = model(img) # forward pass
            # output of model is orderedDict
#             pred = pred['out'] # Batchx21(class)xHxW
            # evaluation
            scores = evaluate_batch(label, pred, cls=cls_num)
            # sum values
            sensitivity_score += scores[0]
            specificity_score += scores[1]
            accuracy_score += scores[2]
            auc_score += scores[3]
            dice_score += scores[4]
    
        # break

    print('{}\t {:.3f}\t\t\t {:.3f}\t\t {:.3f}\t\t {:.3f}\t\t {:.3f}'.format(i ,sensitivity_score/len(val_loader), specificity_score/len(val_loader), accuracy_score/len(val_loader), auc_score/len(val_loader), dice_score/len(val_loader)))
    # append to list (with averaged values over valid set)
    sensitivity_list.append(sensitivity_score/len(val_loader))
    specificity_list.append(specificity_score/len(val_loader))
    accuracy_list.append(accuracy_score/len(val_loader))
    auc_list.append(auc_score/len(val_loader))
    dice_list.append(dice_score/len(val_loader))



In [ ]:
# PLOT
X = range(1, epochs+1)
plt.plot(X, sensitivity_list, label="sensitivity-score")
plt.plot(X, specificity_list, label="specificity-score")
plt.plot(X, accuracy_list, label="accuracy-score")
plt.plot(X, auc_list, label="AUC-ROC score")
plt.plot(X, dice_list, label="Dice coefficient")
plt.xlabel("Number of epochs")
plt.ylabel("Evaluation metrics score")
plt.title("Performance evalaution")
plt.legend() # add legend
plt.savefig('results/T2_eval_metrics.png', bbox_inches='tight')
plt.show()